資料處理（按日期跟航班代碼排序）

In [23]:
import os
import pandas as pd

def load_and_merge_files(start_date, end_date, base_path):
    """
    根據指定的日期範圍自動載入CSV檔案並合併成一個DataFrame，並計算 day left 欄位
    日期格式為 'MMDD'，例如 '1020' 表示10月20日
    """
    # 創建日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2024-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    # 根據日期範圍載入檔案
    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/tokyo_{date}.csv'
        if os.path.exists(file_path):
            # 讀取 CSV 檔案
            df = pd.read_csv(file_path)
            if not df.empty:
                # 清理資料並去除無效行
                df = df.dropna(how="all")  # 刪除全空的行
                # 提取出發日期的 YYYY-MM-DD 部分並格式化
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
                
                # 計算 day left
                file_date = pd.to_datetime(f"2024-{date[:2]}-{date[2:]}")
                df['day left'] = (pd.to_datetime(df['出發日期']) - file_date).dt.days
                
                data_frames.append(df)

    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    return merged_data

def save_csv_with_headers(data, output_path):
    """
    儲存資料到 CSV，在每個不同航班代碼前加入標題行，並清除無效行
    """
    header = list(data.columns)
    previous_flight_code = None

    with open(output_path, 'w', encoding='utf-8-sig') as f:
        # 遍歷每一行資料
        for _, row in data.iterrows():
            # 檢查是否為有效的資料行（非空且包含有效的出發日期）
            if pd.notna(row['出發日期']) and pd.notna(row['航班代碼']):
                # 檢查航班代碼是否變更
                if row['航班代碼'] != previous_flight_code:
                    # 在不同的航班代碼前插入標題行
                    f.write(','.join(header) + '\n')
                    previous_flight_code = row['航班代碼']
                # 寫入資料行
                f.write(','.join([str(item) for item in row]) + '\n')

# 使用範例
base_path = '/Users/yuchingchen/Documents/專題/data'
merged_data = load_and_merge_files('1021', '1110', base_path)

# 按出發日期和航班代碼排序
grouped_data = merged_data.sort_values(by=['出發日期', '航班代碼']).reset_index(drop=True)

# 確保輸出資料夾存在
output_folder = '/Users/yuchingchen/Documents/專題/cleaned_data'
os.makedirs(output_folder, exist_ok=True)

# 將分組後的資料輸出為 CSV 檔案，並在不同航班代碼前插入標題行
output_path = f'{output_folder}/tokyo.csv'
save_csv_with_headers(grouped_data, output_path)
print(f"分組並排序後的資料已輸出到 {output_path}")

分組並排序後的資料已輸出到 /Users/yuchingchen/Documents/專題/cleaned_data/tokyo.csv
